# 02 - Preparação e Mapeamento de Posições

Este notebook realiza:
1. Carregamento dos dados da etapa anterior
2. Aplicação do mapeamento de posições
3. Extração de position, position_group e position_sub_group
4. Validação das posições mapeadas

In [ ]:
import pandas as pd
import yaml
from pathlib import Path

BASE_DIR = Path("c:/jobs/botafogo/v3")
CONFIG_DIR = BASE_DIR / "config"
OUTPUT_DIR = BASE_DIR / "bases" / "outputs"

## 1. Carregar Dados

In [ ]:
# Carregar scouts da etapa anterior
df_scouts = pd.read_parquet(OUTPUT_DIR / "_temp_scouts_raw.parquet")
print(f"Scouts carregados: {len(df_scouts)} jogadores")

# Carregar configuração de posições
with open(CONFIG_DIR / "positions.yaml", "r", encoding="utf-8") as f:
    positions_config = yaml.safe_load(f)

position_mapping = positions_config["position_mapping"]
print(f"Mapeamentos de posição: {len(position_mapping)}")

In [ ]:
# Preencher primary_position nulo com texto padrão
# Jogadores sem posição definida (geralmente baixa minutagem)
df_scouts['primary_position'] = df_scouts['primary_position'].fillna('Sem posição definida')

# Verificar posições originais
print("Posições originais (primary_position):")
print(df_scouts["primary_position"].value_counts())

## 2. Aplicar Mapeamento de Posições

In [ ]:
def map_position(original_position):
    """
    Mapeia uma posição original para a posição padronizada.
    Retorna um dicionário com position, position_group e position_sub_group.
    """
    if pd.isna(original_position) or original_position == "":
        return {"position": None, "position_group": None, "position_sub_group": None}
    
    # Limpar espaços
    original_position = str(original_position).strip()
    
    # Buscar no mapeamento
    if original_position in position_mapping:
        return position_mapping[original_position]
    
    # Tentar case-insensitive
    for key, value in position_mapping.items():
        if key.lower() == original_position.lower():
            return value
    
    # Não encontrado
    return {"position": None, "position_group": None, "position_sub_group": None}

In [ ]:
# Aplicar mapeamento
mapped = df_scouts["primary_position"].apply(map_position)

# Extrair as três colunas
df_scouts["mapped_position"] = mapped.apply(lambda x: x["position"])
df_scouts["position_group"] = mapped.apply(lambda x: x["position_group"])
df_scouts["position_sub_group"] = mapped.apply(lambda x: x["position_sub_group"])

print("Mapeamento aplicado!")

In [ ]:
# Verificar resultado do mapeamento
print("Posições mapeadas (mapped_position):")
print(df_scouts["mapped_position"].value_counts(dropna=False))

In [ ]:
# Verificar position_group
print("\nGrupos de posição (position_group):")
print(df_scouts["position_group"].value_counts(dropna=False))

In [ ]:
# Verificar position_sub_group
print("\nSubgrupos de posição (position_sub_group):")
print(df_scouts["position_sub_group"].value_counts(dropna=False))

## 3. Validação das Posições

In [ ]:
# Posições esperadas (do plano)
EXPECTED_POSITIONS = {"GK", "CB", "RCB", "LCB", "RB", "LB", "DM", "CM", "AM", "LW", "RW", "CF"}

# Verificar posições mapeadas
mapped_positions = set(df_scouts["mapped_position"].dropna().unique())

print(f"Posições esperadas: {sorted(EXPECTED_POSITIONS)}")
print(f"Posições encontradas: {sorted(mapped_positions)}")

# Verificar se todas estão no conjunto esperado
invalid_positions = mapped_positions - EXPECTED_POSITIONS
if invalid_positions:
    print(f"\nPOSIÇÕES INVÁLIDAS: {invalid_positions}")
else:
    print("\nTodas as posições são válidas!")

In [ ]:
# Verificar posições não mapeadas
unmapped = df_scouts[df_scouts["mapped_position"].isna()]
print(f"\nJogadores sem posição mapeada: {len(unmapped)}")

if len(unmapped) > 0:
    print("\nPosições originais não mapeadas:")
    print(unmapped["primary_position"].value_counts())

In [ ]:
# Comparação original vs mapeada
comparison = df_scouts.groupby(["primary_position", "mapped_position"]).size().reset_index(name="count")
print("\nMapeamento completo:")
comparison.sort_values("count", ascending=False)

## 4. Estatísticas por Competição

In [ ]:
# Estatísticas por competição e posição
if "competition_id" in df_scouts.columns:
    stats = df_scouts.groupby(["source_file", "mapped_position"]).size().unstack(fill_value=0)
    print("Distribuição de posições por arquivo:")
    display(stats)

## 5. Salvar Dados Processados

In [ ]:
# Salvar dados com posições mapeadas
df_scouts.to_parquet(OUTPUT_DIR / "_temp_scouts_positions.parquet", index=False)
print(f"Dados salvos: {OUTPUT_DIR / '_temp_scouts_positions.parquet'}")

print(f"\nColunas adicionadas: mapped_position, position_group, position_sub_group")
print(f"Total de jogadores: {len(df_scouts)}")
print(f"Jogadores com posição mapeada: {df_scouts['mapped_position'].notna().sum()}")